In [12]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [13]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [ ]:
# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open("smartwaiver_test").sheet1